# 本文件用于计算训练集中的图片均值、方差，作为测试集的估计

In [1]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image

In [2]:
def getStat(train_data):
    """计算均值方差"""
    print(len(train_data))
    train_loader = torch.utils.data.DataLoader(
        train_data, batch_size=1, shuffle=False, num_workers=0,
        pin_memory=True)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    for _, X in train_loader:
        for d in range(3):
            mean[d] += X[:, d, :, :].mean()
            std[d] += X[:, d, :, :].std()
    mean.div_(len(train_data))
    std.div_(len(train_data))
    return list(mean.numpy()), list(std.numpy())

In [5]:
class myBigBatchDataset_lowMemory(Dataset):
  def __init__(self,data_set_path,train):
    self.labelList=[]
    self.imgList=[]
    trainMap={}
    self.data_set_path=data_set_path

    if train:
      train=1
    else:
      train=0

    with open(data_set_path+r"train_test_split.txt",'r') as trainList:
      trainList=trainList.readlines()
    for line in trainList:
      listt=line.split()
      trainMap[int(listt[0])]=int(listt[1])

    with open(data_set_path+r"images.txt",'r') as imgNames:
      imgNames=imgNames.readlines()

    with open(data_set_path+r"image_class_labels.txt",'r') as image_class_labels:
      image_class_labels=image_class_labels.readlines()

    for i in range(len(trainList)):
      if trainMap[i+1]!=train:
        continue
      imgName=imgNames[i].split()[1]
      image_class_label=int(image_class_labels[i].split()[1])-1
      self.labelList.append(image_class_label)
      self.imgList.append(imgName)

    print(len(self.labelList))
    print(len(self.imgList))


  def __getitem__(self, index):
    imgPath=self.data_set_path+"images/"+self.imgList[index]
    trans=transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
      ])

    op=Image.open(imgPath).convert('RGB')  # 貌似这是jpg的图所以没有透明通道，所以.convert('RGB')与否不重要
    # op=op.resize((224,224))
    op=trans(op)

    return self.labelList[index],op

  def __len__(self):
    return len(self.labelList)

In [7]:
if __name__ == '__main__':
    train_dataset = myBigBatchDataset_lowMemory("./CUB_200_2011/CUB_200_2011/",train=True)
    print(getStat(train_dataset))

5994
5994
5994
([0.4831071, 0.49175683, 0.42481932], [0.18385762, 0.18334596, 0.19432227])


In [ ]:
#test  ([0.4840248, 0.49250537, 0.42385778], [0.18513449, 0.1837308, 0.19505417])
#train ([0.4831071, 0.49175683, 0.42481932], [0.18385762, 0.18334596, 0.19432227])